In [2]:
from huggingface_hub import notebook_login

# notebook_login()

In [ ]:
! pip install transformers datasets evaluate

In [5]:
from datasets import load_dataset, DatasetDict, Dataset

dataset = load_dataset("aslg_pc12")
dataset = dataset["train"].train_test_split(train_size=0.8)

Found cached dataset aslg_pc12 (C:/Users/fuyul/.cache/huggingface/datasets/aslg_pc12/default/0.0.1/7ae5d117644e44ff4d2233f27f1d5df93f429efab7b2ea84828516fcbce48fb4)


  0%|          | 0/1 [00:00<?, ?it/s]

In [17]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("t5-small")

C:\Users\fuyul\AppData\Roaming\Python\Python39\site-packages\transformers\models\t5\tokenization_t5_fast.py:155: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-small automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [19]:
source_lang = "gloss"
target_lang = "text"


def preprocess_function(examples):
    inputs = examples[source_lang]
    targets = examples[target_lang]
    model_inputs = tokenizer(inputs, text_target=targets, max_length=128, truncation=True)
    return model_inputs

In [20]:
tokenized_data = dataset.map(preprocess_function, batched=True)

  0%|          | 0/71 [00:00<?, ?ba/s]

  0%|          | 0/18 [00:00<?, ?ba/s]

In [21]:
! pip install sacrebleu

Defaulting to user installation because normal site-packages is not writeable


In [22]:
import evaluate

sacrebleu = evaluate.load("sacrebleu")

In [23]:
import numpy as np

def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels


def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = sacrebleu.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

In [24]:
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained("t5-small")

ImportError: 
AutoModelForSeq2SeqLM requires the PyTorch library but it was not found in your environment. Checkout the instructions on the
installation page: https://pytorch.org/get-started/locally/ and follow the ones that match your environment.
Please note that you may need to restart your runtime after installation.


In [16]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

In [18]:
training_args = Seq2SeqTrainingArguments(
    output_dir="transformer_model",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=3,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=True,
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_data["train"],
    eval_dataset=tokenized_data["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

Epoch,Training Loss,Validation Loss,Bleu,Gen Len
1,1.157400,0.738817,49.174300,15.653700
2,0.938000,0.618889,54.345600,15.604700
3,0.890900,0.591974,55.673800,15.576200


Saving model checkpoint to transformer_model/checkpoint-13000
Configuration saved in transformer_model/checkpoint-13000/config.json
Model weights saved in transformer_model/checkpoint-13000/pytorch_model.bin
tokenizer config file saved in transformer_model/checkpoint-13000/tokenizer_config.json
Special tokens file saved in transformer_model/checkpoint-13000/special_tokens_map.json
Deleting older checkpoint [transformer_model/checkpoint-11500] due to args.save_total_limit
The following columns in the evaluation set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: text, gloss. If text, gloss are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 17542
  Batch size = 16


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=13158, training_loss=1.1222635127976592, metrics={'train_runtime': 2235.7705, 'train_samples_per_second': 94.153, 'train_steps_per_second': 5.885, 'total_flos': 3206598319669248.0, 'train_loss': 1.1222635127976592, 'epoch': 3.0})

In [19]:
dataset["train"][5]

{'gloss': 'BE X-IT TO BE ARTICLE 215 OR ARTICLE 75 TREATY ON FUNCTION EUROPEAN UNION ?\n',
 'text': 'is it to be article 215 or article 75 of the treaty on the functioning of the european union ?\n'}

In [20]:
text = dataset["train"][5]["gloss"]

In [ ]:
from transformers import pipeline

translator = pipeline("translation", model="transformer_model")

prediction = []
answer = []

def testing_data():
  for i in range(5):
    text = dataset["train"][i]["gloss"]
    answer.append(dataset["train"][i]["text"])
    prediction.append(translator(text))

In [28]:
testing_data()
print(prediction)
print(answer)

[[{'translation_text': 'IF REFERENDUM FAIL , SIR , you will not be thankful by the long-term prosperity of Europe .'}], [{'translation_text': 'ich beabsichtige, daß es in südlich von Lanka zu einer eklatanten Trennung kommt .'}], [{'translation_text': 'es ist nicht mehr ungewöhnlich, als eine ungewöhnliche GRUPPE in gleicher Weise .'}], [{'translation_text': 'X-I notice very carefully how it is done .'}], [{'translation_text': 'ITS implementation will have a large consensus for budgetary power institution and financial implementation .'}]]
['if the referendum fails , sir , you will not be thanked by the vast majority of europeans .\n', 'i therefore wish merely to say that the situation in sri lanka is extremely tragic .\n', 'there is nothing more unfair than treating unequal groups in the same way .\n', 'i noticed very carefully how it was done .\n', 'its implementation will have major consequences for the budgetary powers of the institutions and financial implications .\n']
